<a href="https://colab.research.google.com/github/darshvaghasia12/BoomBoom/blob/master/Book%20Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!unzip "/content/1938_3914_bundle_archive.zip"

Archive:  /content/1938_3914_bundle_archive.zip
replace book_tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
%matplotlib inline


In [13]:
books_df=pd.read_csv('books.csv')
ratings_df=pd.read_csv('ratings.csv')
book_tags=pd.read_csv('book_tags.csv')
tags=pd.read_csv('tags.csv')
to_read=pd.read_csv('to_read.csv')
books_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [7]:
ratings_df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [8]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [9]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [10]:
to_read.head()

,user_id,book_id
0,1,112
1,1,235
2,1,533
3,1,1198
4,1,1874


In [18]:
books_df['title']=books_df['title'].apply(lambda x:x.strip())
books_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [39]:
temp_df=books_df[['book_id','title','average_rating']]
temp_df[0:5]

,book_id,title,average_rating
0,2767052,"The Hunger Games (The Hunger Games, #1)",4.34
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,4.44
2,41865,"Twilight (Twilight, #1)",3.57
3,2657,To Kill a Mockingbird,4.25
4,4671,The Great Gatsby,3.89


In [38]:
userinput=[
           {'title':'The Hunger Games (The Hunger Games, #1','ratings':4},
           {'title':'Harry Potter and the Sorcerer\'s Stone (Harry Potter, #1)','ratings':4.5},
           {'title':'Twilight (Twilight, #1)','ratings':3},
           {'title':'To Kill a Mockingbird','ratings':4.2},
           {'title':'The Great Gatsby','ratings':3.4},
]

input_books=pd.DataFrame(userinput)
input_books



,title,ratings
0,"The Hunger Games (The Hunger Games, #1",4.0
1,Harry Potter and the Sorcerer's Stone (Harry P...,4.5
2,"Twilight (Twilight, #1)",3.0
3,To Kill a Mockingbird,4.2
4,The Great Gatsby,3.4


In [41]:
#Filtering Out the Books by title
input_Id=temp_df[temp_df['title'].isin(input_books['title'].tolist())]
#Then Merging the it so We can get the Book_id
input_books=pd.merge(input_Id,temp_df)
input_books


,book_id,title,average_rating
0,3,Harry Potter and the Sorcerer's Stone (Harry P...,4.44
1,41865,"Twilight (Twilight, #1)",3.57
2,2657,To Kill a Mockingbird,4.25
3,4671,The Great Gatsby,3.89


In [59]:
#Filtering out users that have read books that the input has read and storing it
userSubset1=ratings_df[ratings_df['book_id'].isin(input_books['book_id'].tolist())]
userSubset1.head()

,book_id,user_id,rating
200,3,314,3
201,3,588,1
202,3,2077,2
203,3,2487,3
204,3,2900,3


In [61]:
userSubsetGroup=userSubset1.groupby(['user_id'])

In [72]:
userSubsetGroup.get_group(2077)


,book_id,user_id,rating
202,3,2077,2
265413,2657,2077,1


In [74]:
#sorting it so users with most books in common with the input will have priority
userSubsetGroup=sorted(userSubsetGroup,key=lambda x:len(x[1]),reverse=True)

In [75]:
userSubsetGroup[0:3]

[(2077,         book_id  user_id  rating
  202           3     2077       2
  265413     2657     2077       1), (10610,         book_id  user_id  rating
  222           3    10610       1
  466137     4671    10610       3), (13274,         book_id  user_id  rating
  232           3    13274       5
  265440     2657    13274       3)]

In [76]:
userSubsetGroup=userSubsetGroup[0:100]

In [79]:
pearsonCorrelationDict={}

for name,group in userSubsetGroup:
  group=group.sort_values(by='book_id')
  input_books=input_books.sort_values(by='book_id')
  nRatings=len(group)
  temp1_df=input_books[input_books['book_id'].isin(group['book_id'].tolist())]
  tempRatingList=temp1_df['average_rating'].tolist()
  tempGroupList=group['rating'].tolist()
  Sxx=sum([i**2 for i in tempRatingList])-pow(sum(tempRatingList),2)/float(nRatings)
  Syy=sum([i**2 for i in tempGroupList])-pow(sum(tempGroupList),2)/float(nRatings)
  Sxy=sum(i*j for i,j in zip(tempRatingList,tempGroupList))-sum(tempGroupList)*sum(tempGroupList)/float(nRatings)

  if Sxx!=0 and Syy!=0:
    pearsonCorrelationDict[name]=Sxy/sqrt(Sxx*Syy)
  else:
    pearsonCorrelationDict[name]=0

In [80]:
pearsonCorrelationDict.items()

dict_items([(2077, 90.84210526318778), (10610, 14.745454545454322), (13274, 15.526315789478804), (33716, 15.927272727272497), (36240, 41.473684210539965), (47370, 0), (206, 0), (314, 0), (324, 0), (588, 0), (1182, 0), (1392, 0), (1518, 0), (1600, 0), (1800, 0), (2434, 0), (2487, 0), (2629, 0), (2722, 0), (2779, 0), (2900, 0), (2981, 0), (3261, 0), (3296, 0), (3662, 0), (4124, 0), (4212, 0), (4536, 0), (4706, 0), (4714, 0), (4791, 0), (5115, 0), (5136, 0), (5203, 0), (5272, 0), (5461, 0), (5467, 0), (5624, 0), (5852, 0), (5885, 0), (6008, 0), (6063, 0), (6616, 0), (6630, 0), (6634, 0), (6965, 0), (7001, 0), (7028, 0), (7123, 0), (7225, 0), (7599, 0), (7834, 0), (8501, 0), (8750, 0), (8779, 0), (8861, 0), (9064, 0), (9214, 0), (9246, 0), (9440, 0), (9456, 0), (9608, 0), (9673, 0), (9731, 0), (9771, 0), (9826, 0), (9841, 0), (10140, 0), (10246, 0), (10249, 0), (10455, 0), (10509, 0), (10751, 0), (10944, 0), (10988, 0), (11134, 0), (11285, 0), (11408, 0), (11599, 0), (11675, 0), (11854, 0)

In [81]:
pearsonDF=pd.DataFrame.from_dict(pearsonCorrelationDict,orient='index')
pearsonDF.columns=['similarityIndex']
pearsonDF['user_id']=pearsonDF.index
pearsonDF.index=range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,user_id
0,90.842105,2077
1,14.745455,10610
2,15.526316,13274
3,15.927273,33716
4,41.473684,36240


In [82]:
#Top 50 Users that are most similar to the input
topUsers=pearsonDF.sort_values(by='similarityIndex',ascending=False)[0:50]
topUsers.head()

,similarityIndex,user_id
0,90.842105,2077
4,41.473684,36240
3,15.927273,33716
2,15.526316,13274
1,14.745455,10610


In [83]:
topUsersRating=topUsers.merge(ratings_df,left_on='user_id',right_on='user_id',how='inner')
topUsersRating.head()

,similarityIndex,user_id,book_id,rating
0,90.842105,2077,1,4
1,90.842105,2077,3,2
2,90.842105,2077,6,2
3,90.842105,2077,11,3
4,90.842105,2077,12,3


In [84]:
#Multiplies the similarity by the user's Ratings
topUsersRating['weightedRating']=topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,user_id,book_id,rating,weightedRating
0,90.842105,2077,1,4,363.368421
1,90.842105,2077,3,2,181.684211
2,90.842105,2077,6,2,181.684211
3,90.842105,2077,11,3,272.526316
4,90.842105,2077,12,3,272.526316


In [85]:
tempTopUsersRating=topUsersRating.groupby('book_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns=['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
book_id,,
1,121.514833,516.732057
2,30.672727,153.363636
3,178.514833,404.790431
4,15.927273,63.709091
5,15.927273,63.709091


In [88]:
recommendation_df=pd.DataFrame()
recommendation_df['weighted average recommendation score']=tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['book_id']=tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,book_id
book_id,,
1,4.252420,1
2,5.000000,2
3,2.267545,3
4,4.000000,4
5,4.000000,5


In [89]:
recommendation_df=recommendation_df.sort_values(by='weighted average recommendation score',ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,book_id
book_id,,
31,5.0,31
2,5.0,2
179,5.0,179
1292,5.0,1292
427,5.0,427
858,5.0,858
1202,5.0,1202
862,5.0,862
863,5.0,863


In [90]:
temp_df.loc[temp_df['book_id'].isin(recommendation_df.head(10)['book_id'].tolist())]

,book_id,title,average_rating
20,2,Harry Potter and the Order of the Phoenix (Har...,4.46
91,1202,Freakonomics: A Rogue Economist Explores the H...,3.93
